In [1]:
!pip install pydicom

In [3]:
import pydicom
pydicom.dcmread("C:/Users/James/Documents/Head-Neck-PET-CT/manifest-VpKfQUDr2642018792281691204/Head-Neck-PET-CT/HN-CHUM-022/08-27-1885-NA-ORL TOMO C PROT TEP-32933/4.000000-CTnormalFOV500-49859/1-028.dcm")

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 194
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.5168.2407.253171656809491354212746663369
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.35'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'HELIX']
(0008, 0012) Instance Creation Date              DA: '18850827'
(0008, 0013) Instance Creation Time              TM: '111111'
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) 